# クイックスタート4

このノートのクイックスタートは、Microsoft の Azure AI Foundry を使って 推論チャットの結果を評価するアプリケーションの開発を行うための学習ノートです。

※この内容は 2025年2月時点での仕様等に基づくものです。仕様やドキュメントの変更に伴い、この内容が古くなる可能性があります。

## 前提条件

このノートの内容を学習するにあたって、以下が必要になります。

- Visial Studio Code または このノートブックを実行可能な Jupyter が動作する環境
- Azure サブスクリプション
- 作成済みの Azure AI Foundry プロジェクトとデプロイしたモデル
- Azure AI Foundry プロジェクトに接続済みの Application Insights インスタンス（トレースに利用） 
- Python 3.8 以降
- Python に関する基礎知識
- 必要な Python パッケージ（下記のコードでインストールする）


In [ ]:
# 必要な Python パッケージのインストール
%pip install python-dotenv azure-identity azure-ai-projects azure-ai-inference azure-ai-evaluation azure-ai-ml azure-search-documents azure-core-tracing-opentelemetry azure-monitor-opentelemetry opentelemetry-sdk

## Azure AI Foundry で 推論モデルの結果に対する評価を行うアプリを開発する

このノートでは、Azure AI Foundry SDK を使用して、推論モデルを使用したチャットアプリケーションのクエリ・結果等に対する評価を行うアプリケーションを開発する方法について説明します。

AI の評価は、AI アプリケーションの導入効果を最大化し、ビジネスにおける AI 活用を成功に導くためｎに対する信頼と信用を築くための 生成 AI アプリケーションのライフサイクルの不可欠な要素です。AI アプリケーションが、コンテキストに基づかない、無関係または一貫性のない、捏造された出力を生成する可能性があると、その結果、アプリケーションのエクスペリエンスが悪化したり、誤った情報に基づく決断がなされたり、組織が悪意のある攻撃にさらされたり、その他のさまざまな悪影響が生じたりする可能性があります。AIアプリケーションの品質評価や信頼性評価が重要となります。

`azure.ai.projects` と `azure.ai.evaluation` のライブラリを使用して、評価アプリケーションを開発します。
ここで開発するアプリケーションは、事前に準備済みの評価データセットをアップロードして、クラウド（リモート）で評価を行います。
評価はローカルで実施することもできます。


### 準備

このクイック スタートを始める前に、Azure AI Foundry ポータルから、ハブ リソースとプロジェクト リソースを作成して `gpt-4o` または `gpt-4o-mini` モデルをプロジェクトへデプロイしてください。

エージェントをサポートするモデルには制限があります。[技術ドキュメント](https://learn.microsoft.com/ja-jp/azure/ai-services/agents/concepts/model-region-support?tabs=python) を参照して、適切なモデルとバージョンのエージェントをデプロイします。

> [Azure AI Foundry プレイグラウンドのクイックスタート](https://learn.microsoft.com/ja-jp/azure/ai-studio/quickstarts/get-started-playground) を完了している場合、必要な作業は完了しています。

Visual Studio Code でこのノートを開き、[ドキュメントのこのセクション](https://code.visualstudio.com/docs/python/environments#_creating-environments) を参考にしながら、ワークスペースに Python 仮想環境を作成します。仮想環境を作成する際には、このフォルダにある `requirements.txt` を選択すると、このノートの Python アプリケーションコードの実行に必要なライブラリを Python 仮想環境にインストールすることができます。

仮想環境の準備ができたら、このノートにある Python コードをステップごとに実行します。


### チャットアプリのコードサンプル

#### 1. AI プロジェクトに接続します

[クイックスタート3](../03_rag_chat/rag_chat.ipynb) で実行した手順と同様に、AI プロジェクトに接続し、SDK のトレースを有効にします。


In [ ]:
import os
from dotenv import load_dotenv
from opentelemetry import trace
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential

tracer = trace.get_tracer(__name__)

load_dotenv('.env', override=True)
project_connection_string = os.getenv('PROJECT_CONNECTION_STRING')
model_name_string = os.getenv('MODEL_NAME')
model_api_version_string = os.getenv('MODEL_API_VERSION')

try:
    credential = DefaultAzureCredential()
    print(f"✅ Azure 資格情報の初期化が成功しました.")
except Exception as e:
    print(f"❌ Azure 資格情報の初期化が以下の理由で失敗しました: {str(e)}")

try:
    project = AIProjectClient.from_connection_string(
        conn_str=project_connection_string,
        credential=credential
    )
    print(f"✅ プロジェクトクライアントの初期化が成功しました.")
except Exception as e:
    print(f"❌ プロジェクトクライアントの初期化が以下の理由で失敗しました: {str(e)}")

以下のコードを実行して、トレースを有効にします。


In [ ]:
from azure.ai.inference.tracing import AIInferenceInstrumentor
from azure.monitor.opentelemetry import configure_azure_monitor
from azure.core.settings import settings

# Enable Azure SDK tracing with either of two lines:
os.environ["AZURE_SDK_TRACING_IMPLEMENTATION"] = "opentelemetry"
settings.tracing_implementation = "opentelemetry"

# Enable tracing
AIInferenceInstrumentor().instrument()
# enable logging message contents
os.environ["AZURE_TRACING_GEN_AI_CONTENT_RECORDING_ENABLED"] = "true"

application_insights_connection_string = project.telemetry.get_connection_string()
if not application_insights_connection_string:
    print("❌ Application Insights がこのプロジェクトで有効になっていません.")
    print("==> Azure AI Foundry ポータルの \"トレース\"タブから、Application Inights を有効にしてください.")
else:
    configure_azure_monitor(connection_string=application_insights_connection_string)
    print("✅ Application Insights によるトレースを有効にしました.")

#### 2. 評価データセットのアップロードを行います

リモート評価で利用する評価用のデータセットが含まれたファイルを AI プロジェクトにアップロードします。

評価用のデータは、評価する項目に合わせて、JSON リスト形式で以下の内容を用意します。個々のデータで用意するプロパティは以下の内容となります。
- `query`: ユーザーがシステムに対して行う質問やクエリを指します
- `context`: システムが応答を生成する際に利用する、関連情報や背景情報を指します
- `response`: システムが `query` と `context` に基づいて生成した応答、回答、または、出力を指します
- `ground_truth`: `query` に対する理想的な正解となる回答や模範解答を指します


In [ ]:

@tracer.start_as_current_span("quick_start_04: upload_evaluation_file")
def upload_evaluation_file():
  # Upload data for evaluation and get dataset id
  data_id, _ = project.upload_file("./data/eval_data_ja.jsonl")
  return data_id

data_id = upload_evaluation_file()

#### 3. 評価項目を定義する


Azure AI Foundry で対応する AI 品質評価は以下の通りです。

リモート評価で評価する項目を決めて定義します。ここでは、以下の項目について、品質評価を指定します。


- AI 品質 (AI 支援)

    評価項目は以下の通りです。

    | 項目 | クラス | 説明 |
    | --- | -- | -- |
    | 根拠性 | GroundednessEvaluator | 生成 AI アプリケーションで生成された回答が、入力ソースからの情報とどの程度一致しているかを計測します。 |
    | 関連性 | RelevanceEvaluator | 生成 AI アプリケーションで生成された回答がどの程度適切で、提示された質問に直接関連するかを計測します。 |
    | コヒーレンス | CoherenceEvaluator | 生成 AI アプリケーションが、スムーズに流れ、自然に読み取られ、人間のような言語に似た出力を生成できる程度を測定します。 |
    | 流暢性 | FluencyEvaluator | 生成 AI アプリケーションの予測応答の言語習熟度を測定します。 |
    | 類似性 | SimilarityEvaluator | ソース データ (グラウンド トゥルース) 文と生成 AI アプリケーションで生成された応答の間の類似性を計測します。 |
    
    評価データに必要となるプロパティは以下の通りです。
    
    | 項目 | query | context | response | ground_truth |
    | --- | -- | -- | -- | -- |
    | 根拠性      | ✅ | ✅ | ✅ |  |
    | 関連性      | ✅ |  | ✅ |  |
    | コヒーレンス | ✅ |  | ✅ |  |
    | 流暢性      |  |  | ✅ |  |
    | 類似性      | ✅ |  | ✅ | ✅ |



In [ ]:
from azure.ai.projects.models import ConnectionType
from azure.ai.projects.models import EvaluatorConfiguration
from azure.ai.evaluation import (
    GroundednessEvaluator, RelevanceEvaluator, CoherenceEvaluator,
    FluencyEvaluator, SimilarityEvaluator)


@tracer.start_as_current_span("quick_start_04: get_evaluators")
def get_evaluators():
    default_connection = project.connections.get_default(
        connection_type=ConnectionType.AZURE_OPEN_AI
    )
    model_config = default_connection.to_evaluator_model_config(
        deployment_name=model_name_string, api_version=model_api_version_string, include_credentials=True,
    )
    evaluators={
        "groundedness": EvaluatorConfiguration(id=GroundednessEvaluator.id, init_params={"model_config": model_config}),
        "relevance": EvaluatorConfiguration(id=RelevanceEvaluator.id, init_params={"model_config": model_config}),
        "coherence": EvaluatorConfiguration(id=CoherenceEvaluator.id, init_params={"model_config": model_config}),
        "fluency": EvaluatorConfiguration(id=FluencyEvaluator.id, init_params={"model_config": model_config}),
        "similarity": EvaluatorConfiguration(id=SimilarityEvaluator.id, init_params={"model_config": model_config}),
    }
    return evaluators

evaluators = get_evaluators()


#### 4. AI プロジェクトで評価を実行します


アップロードした評価データセットに対して、定義した評価項目で、AI プロジェクト上で評価を実行します。


In [ ]:
import time
from azure.ai.projects.models import Evaluation, Dataset

@tracer.start_as_current_span("quick_start_04: run_remote_evaluation")
def run_remote_evaluation(data_id, evaluators):
    evaluation = Evaluation(
        display_name=f"リモート評価 - {time.strftime("%Y%m%d%H%M%S")}",
        description=f"リモート評価 - {time.strftime("%Y%m%d%H%M%S")}: サンプルデータセットの評価テスト",
        data=Dataset(id=data_id),
        evaluators=evaluators,
    )
    evaluation_response = project.evaluations.create(evaluation=evaluation)
    return evaluation_response

eval_response = run_remote_evaluation(data_id, evaluators)


#### 5. 評価の実行結果を取得します。


評価結果は、Azure AI Foundry ポータルで閲覧することができます。
また、評価結果のレポートを取得することもできます。


In [ ]:
print(eval_response)

get_evaluation_response = project.evaluations.get(eval_response.id)
print("------------------------------------------------------------------------")
print("Created evaluation, evaluation ID: ", get_evaluation_response.id)
print("Evaluation status: ", get_evaluation_response.status)
if isinstance(get_evaluation_response.properties, dict):
    print("AI Foundry URI: ", get_evaluation_response.properties["AiStudioEvaluationUri"])
print("------------------------------------------------------------------------")